In [25]:
import pandas as pd
import numpy as np
from scipy import sparse
from lightfm import LightFM
import numpy as np
import pandas as pd 

In [26]:
PATH = r"D:\Projects\moveilens"
df_rating = pd.read_csv(fr"{PATH}/rating.csv")
df_rating = df_rating.iloc[:50000]
df_rating.head()

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39
3,1,47,3.5,2005-04-02 23:32:07
4,1,50,3.5,2005-04-02 23:29:40


In [ ]:
df_movies = pd.read_csv(fr'{PATH}/movie.csv')
df_movies.head()

In [31]:
# Setting the rows to users
user_item = df_rating.groupby(['userId', 'movieId'])['rating'].sum().unstack()
user_item = user_item.reset_index().fillna(0)
user_item = user_item.set_index('userId')
user_item.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,115149,115569,115617,116797,117176,117511,117590,118696,125916,128488
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
def create_user_dict(interactions):
    user_id = list(interactions.index)
    user_dict = {}
    counter = 0 
    for i in user_id:
        user_dict[i] = counter
        counter += 1
    return user_dict

def create_item_dict(df,id_col,name_col):
    item_dict ={}
    for i in range(df.shape[0]):
        item_dict[(df.loc[i,id_col])] = df.loc[i,name_col]
    return item_dict

In [13]:
# Create User Dict
user_dict = create_user_dict(interactions=interactions)
# Create Item dict
movies_dict = create_item_dict(df = df_movies,
                               id_col = 'movieId',
                               name_col = 'title')

In [32]:
# Hyperparams
LATENT_FACTORS = 25
LOSS  ='warp'
K_POSITIVE = 3
EPOCH = 50
THREADS = 2

In [33]:
# Training
train = sparse.csr_matrix(user_item.values)
hybrid_model = LightFM(no_components=LATENT_FACTORS, loss=LOSS, k=K_POSITIVE)
hybrid_model.fit(train, epochs=EPOCH, num_threads=THREADS)

In [22]:
def sample_recommendation_user(model, user_item, user_id, user_dict, item_dict, threshold, nrec_items):
    n_users, n_items = interactions.shape
    user_x = user_dict[user_id]
    scores = pd.Series(model.predict(user_x,np.arange(n_items)))
    scores.index = interactions.columns
    scores = list(pd.Series(scores.sort_values(ascending=False).index))
    
    known_items = list(pd.Series(interactions.loc[user_id,:][interactions.loc[user_id,:] > threshold].index).sort_values(ascending=False))
    
    scores = [x for x in scores if x not in known_items]
    return_score_list = scores[0:nrec_items]
    known_items = list(pd.Series(known_items).apply(lambda x: item_dict[x]))
    scores = list(pd.Series(return_score_list).apply(lambda x: item_dict[x]))
    print("\n Recommendations:")
    counter = 1
    for i in scores:
        print(str(counter) + '- ' + i)
        counter+=1
    return return_score_list

In [24]:
# Recommendation for random user
import random
random_user = random.randint(1, 100)

recommendations = sample_recommendation_user(model = hybrid_model, 
                                      user_item = user_item, 
                                      user_id = random_user, 
                                      user_dict = user_dict,
                                      item_dict = movies_dict, 
                                      threshold = 4,
                                      nrec_items = 5,
                                      show = True)

Known Likes:
1- Jaws (1975)
2- Graduate, The (1967)
3- Godfather: Part II, The (1974)
4- Godfather, The (1972)
5- Schindler's List (1993)

 Recommended Items:
1- Star Wars: Episode V - The Empire Strikes Back (1980)
2- Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)
3- Star Wars: Episode IV - A New Hope (1977)
4- Fargo (1996)
5- One Flew Over the Cuckoo's Nest (1975)
